In [ ]:
#use review data
x_r = range(2015,2021)
y1r =[100*sum(philly[philly.most_recent_review.dt.year==y].is_cbd)/len(philly[philly.most_recent_review.dt.year==y])
        for y in x_r]
y2r = [100*sum(philly[philly.earliest_review.dt.year==y].is_cbd)/len(philly[philly.earliest_review.dt.year==y])
        for y in x_r]
fig = plt.figure(figsize=(13,5))
ax = fig.add_subplot(121)
ax_o = fig.add_subplot(122)
ax.bar(x_r,[len(philly[philly.most_recent_review.dt.year==y]) for y in x_r],color="tab:cyan", width= 0.5 )
ax.bar(x_r,[sum(philly[philly.most_recent_review.dt.year==y].is_cbd) for y in x_r],color="tab:blue", width= 0.5 )
ax.set_xlabel("Closed",fontsize=14)
ax.set_ylabel("Business Count",fontsize=14)
#ax.set_ylim(0,1200)
ax2=ax.twinx()
ax2.plot(x_r,y1r, color="tab:red",marker="o" )
ax2.set_ylim(0,100)
ax.set_xticks(x_r, minor=False)
ax_o.bar(x_r,[len(philly[philly.earliest_review.dt.year==y]) for y in x_r],color="tab:cyan", width= 0.5 )
ax_o.bar(x_r,[sum(philly[philly.earliest_review.dt.year==y].is_cbd) for y in x_r],color="tab:blue", width= 0.5 )
ax_o.set_xlabel("Open",fontsize=14)
#ax_o.set_ylim(0,1200)
ax2=ax_o.twinx()
ax2.plot(x_r,y2r, color="tab:red",marker="o" )
ax2.set_ylabel("Percentage of CBD (%)",fontsize=14)
ax2.set_ylim(0,100)
ax_o.set_xticks(x_r, minor=False)

fig.suptitle(f'Change in Food Business Count ')
fig.supxlabel('Year')
plt.show()

In [ ]:
review_json_path = '../data/yelp_dataset/yelp_academic_dataset_review.json'
reviews = []
r_dtypes = {"stars": np.float16}
with open(review_json_path, "r") as f:
    reader = pd.read_json(f, orient="records", lines=True, 
                          dtype=r_dtypes, chunksize=1000)
        
    for chunk in tqdm(reader):
        reduced_chunk = chunk.drop(columns=['review_id', 'user_id'])\
                             .query("`date` >= '2015-12-01'")
        reviews.append(reduced_chunk)
    
reviews = pd.concat(reviews, ignore_index=True)

reviews_time = reviews.groupby(['business_id']).agg({'date': [np.min,np.max]}).droplevel(axis=1, level=0).reset_index()
reviews_time.rename({'amin':'earliest_review', 'amax':'most_recent_review'},axis=1,inplace=True)

In [ ]:
def define_status(row,status):
    l = []
    for y in years[1:]:
        if row[status+'_%s'%(y)] ==1:
            l.append()
            return str(y)
        else:
            continue
       
for s in ['close','open']:
    philly[s] = philly.apply(lambda x: define_status(x,s),axis=1)

In [ ]:
#use business license data
x = years[1:]
def all(status,y):
    return len(philly[philly[status]==str(y)])
def cbd(status,y):
    return sum(philly[philly[status]==str(y)].is_cbd)

y1_all = [all('close',y) for y in x]
y1_cbd = [cbd('close',y) for y in x]
y2_all = [all('open',y) for y in x]
y2_cbd = [cbd('open',y) for y in x]

y1 =[100*cbd('close',y)/all('close',y) for y in x]
y2 = [100* cbd('open',y)/all('open',y)  for y in x]

fig = plt.figure(figsize=(13,5))
ax = fig.add_subplot(121)
ax_o = fig.add_subplot(122)
ax.bar(x,y1_all,color="tab:cyan", width= 0.5 )
ax.bar(x,y1_cbd,color="tab:blue", width= 0.5 )

ax.set_xlabel("Closed",fontsize=14)
ax.set_ylabel("Business Count",fontsize=14)
ax.set_ylim(0,1800)
ax2=ax.twinx()
ax2.plot(x,y1, color="tab:red",marker="o" )
ax2.set_ylim(20,50)

ax.set_xticks(x, minor=False)
ax_o.bar(x,y2_all,color="tab:cyan", width= 0.5 )
ax_o.bar(x,y2_cbd,color="tab:blue", width= 0.5 )
ax_o.set_xlabel("New Open",fontsize=14)
ax_o.set_ylim(0,1800)
ax2=ax_o.twinx()
ax2.plot(x,y2, color="tab:red",marker="o" )
ax2.set_ylabel("Rate of Businesses in CBD (%)",fontsize=14)
ax2.set_ylim(20,50)
ax_o.set_xticks(x, minor=False)
ax.vlines(2019, 0, 4000, linestyles='--', color='b')
ax_o.vlines(2019, 0, 4000, linestyles='--', color='b')

fig.suptitle(f'Change in all Business Count')
fig.supxlabel('Year')
plt.savefig("../output/licenses.png")
plt.show()

In [ ]:
def gen_census(row):
    map = cg.coordinates(x=row.longitude , y=row.latitude)
    return map['2018 State Legislative Districts - Upper'][0]['GEOID']
philly['census_track']= philly.apply(gen_census, axis=1)

In [ ]:
cor = list(zip(philly.longitude,philly.latitude))
cen = []
for co in cor:
    info = cg.coordinates(x=co[0], y=co[1])
    code = info['2018 State Legislative Districts - Upper'][0]['GEOID']
    cen.append(code)

In [ ]:
df_tract['is_cbd'] = 0
d = {}
for c in df_tract.tract.unique():
    r = df_yelp[df_yelp.tract==c].is_cbd.sum()/len(df_yelp[df_yelp.tract==c].is_cbd)
    if r> 0.5:
        d[c] = 1
    else:
        d[c] = 0

df_tract['is_cbd'] = df_tract.apply(lambda x: d[x.tract], axis=1)
df_tract['post_covid'] = df_tract.apply(lambda x: 1 if x.year>=2020 else 0, axis=1)